In [1]:
!pip install -qU \
  pinecone-client==3.1.0 \
  pinecone-datasets==0.7.0 \
  sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 46.2 MB/s 

In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Load the JSON file into a DataFrame
dataset = pd.read_json("combine101.json")

# Extract the 'content' column
content = dataset['content']

# Display the first few rows of the content column
print(content.head())

# Initialize the model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Define a batch size for encoding
batch_size = 100 # Adjust based on your system's memory capacity

# Initialize an empty list to store the encoded vectors
encoded_vectors = []

# Encode the texts in batches to manage memory usage
for i in range(0, len(dataset), batch_size):
    batch = dataset.iloc[i:i+batch_size]['content'].tolist() # Convert the batch to a list
    vectors = model.encode(batch)
    encoded_vectors.extend(vectors.tolist())

# At this point, `encoded_vectors` contains the encoded vectors for all texts
# You can now proceed with using these vectors as needed



0    08 december 00.31, Skadegörelse, Sigtuna | Pol...
1    26 oktober 09:03, Trafikkontroll, Västerbotten...
2    11 juli 12:35, Arbetsplatsolycka, Burlöv | Pol...
3    Ombud inför EUIPO Du som under minst fem år ha...
4    Jönköping Väg 1819, Mullsjö-Habo, gång- och cy...
Name: content, dtype: object


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/968 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

unigram.json:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [4]:
print(len(encoded_vectors))

25027


In [5]:
import os

use_serverless = os.environ.get("USE_SERVERLESS", "False").lower() == "true"

In [6]:
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pc.io)
api_key = os.environ.get('PINECONE_API_KEY') or '041f5d83-b9ec-4fbd-bb68-f4ba876e82b6'
environment = os.environ.get('PINECONE_ENVIRONMENT') or 'gcp-starter'

# configure client
pc = Pinecone(api_key=api_key)

In [7]:
from pinecone import ServerlessSpec, PodSpec

if use_serverless:
    spec = ServerlessSpec(cloud='aws', region='us-west-2')
else:
    spec = PodSpec(environment=environment)

In [8]:
index_name = 'semantic-search-fast'

In [9]:
import time

existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=384,  # dimensionality of minilm
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0137,
 'namespaces': {'': {'vector_count': 1370}},
 'total_vector_count': 1370}

In [11]:
from tqdm.auto import tqdm

# Prepare the data for upsert with reduced metadata size
records = [(str(i), vector, {'content': item['content'][:100], 'title': item['title'], 'url': item['url']}) for i, (vector, item) in enumerate(zip(encoded_vectors, dataset.to_dict('records')))]

# Upsert the data in batches
batch_size = 128
for batch_start in tqdm(range(0, len(records), batch_size), total=len(records) // batch_size + 1):
    batch_end = min(batch_start + batch_size, len(records))
    batch_records = records[batch_start:batch_end]
    index.upsert(vectors=batch_records)



  0%|          | 0/196 [00:00<?, ?it/s]

In [12]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', device=device)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [33]:
query = "vad är polisens öppettider i växjö?"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(vector=xq, top_k=10, include_metadata=True)
xc

{'matches': [{'id': '74',
              'metadata': {'content': 'OmrÃ¥despoliser â\x80\x93 arbetar '
                                      'lokalt och brottsfÃ¶rebyggande | '
                                      'Polismyndigheten GÃ¥ direkt till innehÃ',
                           'title': '/aktuellt/nyheter/2020/februari/omradespoliser--arbetar-lokalt-och-brottsforebyggande/',
                           'url': 'https://polisen.se/aktuellt/nyheter/2020/februari/omradespoliser--arbetar-lokalt-och-brottsforebyggande/'},
              'score': 7.9308629,
              'values': []},
             {'id': '7155',
              'metadata': {'content': 'Polisregion Nord välkomnar 72 nya '
                                      'poliser idag | Polismyndigheten Gå '
                                      'direkt till innehåll Gå direkt ',
                           'title': '/aktuellt/nyheter/nord/20232/december/polisregion-nord-valkomnar-72-nya-poliser-idag/',
                           'url': '